# Active Appeals Base Build (Silver Layer)
<table style='float:left;'>
   <tbody>
      <tr>
         <td style='text-align: left;'><b>Name: </b></td>
         <td>SILVER_ACTIVE_APPEALS</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>Description: </b></td>
         <td>Notebook responsible for the silver layer, common for all active appeal states.</td>
      </tr>
      <tr>
         <td style='text-align: left;'><b>First Created: </b></td>
         <td>May-2025</td>
      </tr>
      <tr>
         <th style='text-align: left;'><b>Changelog (JIRA ref/initials/date):</b></th>
         <th>Comments</th>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-635">ARIADM-635</a>/NSA/JUN-2025</td>
         <td>Active CCD MVP Base Build: Create Segmentation (Part One) Table - Perform All Joins for CCD-IAC Base Dataset</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-637">ARIADM-637</a>/NSA/JUN-2025</td>
         <td>Active CCD MVP Base Build: Enhance CCD-IAC Segmentation (Part Two) Table - Apply Filters and Case Logic</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-636">ARIADM-636</a>/NSA/JUN-2025</td>
         <td>Active CCD MVP Base Build: Silver Tables with Reference Data Enrichment</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-738">ARIADM-738</a>/NSA/JUL-2025</td>
         <td>Update PySpark of CCD segmentation query</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-847">ARIADM-847</a>/NSA/JUL-2025</td>
         <td>ARIADM-733 Data mismatch in table silver_appealcase_detail : Column : lu_caseManagementLocation </td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-849">ARIADM-849</a>/NSA/JUL-2025</td>
         <td>ARIADM-733 Data mismatch in table silver_status_detail : Column : Notes</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-xxx">ARIADM-xxx</a>/NSA/AUG-2025</td>
         <td>Deduplicate M1: Aggregate CategoryIds and CaseFeeSummary per CaseNo</td>
      </tr>
      <tr>
         <td style='text-align: left;'><a href="https://tools.hmcts.net/jira/browse/ARIADM-657">ARIADM-657</a>/NSA/AUG-2025</td>
         <td>segmentation logic filter fix with PR #258</td>
      </tr>
   </tbody>
</table>

### Import packages

In [0]:
import dlt
import json
# from pyspark.sql.functions import when, col,coalesce, current_timestamp, lit, date_format
from pyspark.sql.functions import *

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env_name = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

print(f"env_code: {lz_key}")  # This won't be redacted
print(f"env_name: {env_name}")  # This won't be redacted

KeyVault_name = f"ingest{lz_key}-meta002-{env_name}"
print(f"KeyVault_name: {KeyVault_name}") 

In [0]:
# Service principal credentials
client_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-ID")
client_secret = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-CLIENT-SECRET")
tenant_id = dbutils.secrets.get(KeyVault_name, "SERVICE-PRINCIPLE-TENANT-ID")

# Storage account names
curated_storage = f"ingest{lz_key}curated{env_name}"
checkpoint_storage = f"ingest{lz_key}xcutting{env_name}"
raw_storage = f"ingest{lz_key}raw{env_name}"
landing_storage = f"ingest{lz_key}landing{env_name}"

# Spark config for curated storage (Delta table)
spark.conf.set(f"fs.azure.account.auth.type.{curated_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{curated_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{curated_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{curated_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{curated_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{checkpoint_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{checkpoint_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{checkpoint_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{checkpoint_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{checkpoint_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{raw_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{raw_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{raw_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{raw_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{raw_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# Spark config for checkpoint storage
spark.conf.set(f"fs.azure.account.auth.type.{landing_storage}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{landing_storage}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{landing_storage}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{landing_storage}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{landing_storage}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

## Set Variables

In [0]:
# read_hive = False

# Setting variables for use in subsequent cells
raw_path = f"abfss://raw@ingest{lz_key}raw{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
landing_path = f"abfss://landing@ingest{lz_key}landing{env_name}.dfs.core.windows.net/SQLServer/Sales/IRIS/dbo/"
bronze_path = f"abfss://bronze@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
silver_path = f"abfss://silver@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
gold_path = f"abfss://gold@ingest{lz_key}curated{env_name}.dfs.core.windows.net/ARIADM/ACTIVE/CCD/APPEALS"
gold_outputs = "ARIADM/CCD/APPEALS"
hive_schema = "ariadm_ccd_apl"
# key_vault = "ingest00-keyvault-sbox"

html_path = f"abfss://html-template@ingest{lz_key}landing{env_name}.dfs.core.windows.net/"

# Print all variables
variables = {
    # "read_hive": read_hive,
    "raw_path": raw_path,
    "landing_path": landing_path,
    "bronze_path": bronze_path,
    "silver_path": silver_path,
    "gold_path": gold_path,
    "html_path": html_path,
    "gold_outputs": gold_outputs,
    "hive_schema": hive_schema,
    "key_vault": KeyVault_name
}

display(variables)

## Segmentation DLT Tables Creation - stg_segmentation_states 


In [0]:
@dlt.table(
    name="stg_balance_due",
    comment="DLT table for balance due per CaseNo based on specific transaction rules.",
    path=f"{silver_path}/stg_balance_due"
)
def stg_balance_due():
    # Read transaction and transaction type data
    try:
        tx = dlt.read("raw_transaction")
        tx_type = dlt.read("raw_transactiontype")
    except:
        # Fallback for interactive runs (debug mode)
        tx = spark.table("ariadm_arm_appeals.raw_transaction")
        tx_type = spark.table("ariadm_arm_appeals.raw_transactiontype")

    # Filter for balance-related transactions
    filtered_tx = (
        tx.join(tx_type, tx.TransactionTypeId == tx_type.TransactionTypeID, "left")
          .filter((tx_type.SumBalance == 1) & (~tx.TransactionId.isin(
              tx.filter(tx.TransactionTypeId.isin([6, 19]))
                .select("ReferringTransactionId")
                .rdd.flatMap(lambda x: x).collect()
          )))
    )

    # Aggregate by CaseNo
    return (
        filtered_tx
            .groupBy("CaseNo")
            .agg(sum("Amount").alias("BalanceDue"))
    )

In [0]:
@dlt.table(
    name="stg_representation",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/stg_representation"
)
def stg_representation():
    try:
        # Read the AppealCase and CaseRep DLT tables
        appeal_case = dlt.read("raw_appealcase").filter(col("CaseType") == 1)
        case_rep = dlt.read("raw_caserep")
    except:
        appeal_case = spark.table("ariadm_active_appeals.raw_appealcase").filter(col("CaseType") == 1)
        case_rep = spark.table("ariadm_active_appeals.raw_caserep")
     
    df = appeal_case.join(case_rep, on="CaseNo", how="left")
        
    # Clean logic for determining Representation
    aip_conditions = (
        (col("RepresentativeID") == 957) |  # RepresentativeID is 957
        (col("RepresentativeID").isNull()) |  # RepresentativeID is NULL
        col("Name").like("%no rep%") |  # Name contains 'no rep'
        col("Name").like("%no  rep%") |  # Name contains 'no  rep'
        col("Name").like("%No rep%") |  # Name contains 'No rep'
        col("Name").like("%No Rep%") |  # Name contains 'No Rep'
        col("Name").like("%NO REP%") |  # Name contains 'NO REP'
        col("Name").like("none") |  # Name contains 'none'
        col("Name").like("%unrepresented%") |  # Name contains 'unrepresented'
        col("Name").like("%self%") |  # Name contains 'self'
        col("Name").like("%not rep%") |  # Name contains 'not rep'
        col("Name").like("%not  rep%") |  # Name contains 'not  rep'
        (col("Name") == "-") |  # Name is '-'
        col("Name").like("%(spo%") |  # Name contains '(spo'
        col("Name").like("%spons%")  # Name contains 'spons'
    )

    return df.select(
        col("CaseNo"),
        when(aip_conditions, "AIP").otherwise("LR").alias("Representation")
    )

In [0]:
from pyspark.sql.functions import when, max as spark_max, add_months
from pyspark.sql import functions as F

# from pyspark.sql.functions import col, current_date, add_months, add_years

@dlt.table(
    name="stg_segmentation_states",
    comment="Bronze DLT table replicating the AppealCase join logic.",
    path=f"{silver_path}/stg_segmentation_states"
)
def stg_segmentation_states():
    # Base tables
    try:
        # Base tables
        appealcase = dlt.read("raw_appealcase")
        status = dlt.read("raw_status")
        caselist = dlt.read("raw_caselist")
        documentsreceived = dlt.read("raw_documentsreceived")
        filelocation = dlt.read("raw_filelocation")
        balancedue = dlt.read("stg_balance_due")
        representation = dlt.read("stg_representation")
    except:
        appealcase = spark.table("ariadm_active_appeals.raw_appealcase")
        status = spark.table("ariadm_active_appeals.raw_status")
        caselist = spark.table("ariadm_active_appeals.raw_caselist")
        documentsreceived = spark.table("ariadm_active_appeals.raw_documentsreceived")
        filelocation = spark.table("ariadm_active_appeals.raw_filelocation")
        balancedue = spark.table("ariadm_active_appeals.stg_balance_due")
        representation = spark.table("ariadm_active_appeals.stg_representation")
                  
    s_subquery01 = (
        status.filter(((~col("outcome").isin([38, 111]))) &
                  ((~col("casestatus").isin(['17']))))
          .groupBy("CaseNo")
          .agg(spark_max("StatusId").alias("max_ID"))
    )
                  
    # Previous status logic
    prev_subquery02 = (
        status.filter((col("casestatus").isNull()) | (~col("casestatus").isin(['50', '52', '36'])))
          .groupBy("CaseNo")
          .agg(spark_max("StatusId").alias("Prev_ID"))
    )

    # UT status
    ut_subquery03 = (
        status.filter(col("CaseStatus").isin(['40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33']))
          .groupBy("CaseNo")
          .agg(spark_max("StatusId").alias("UT_ID"))
    )

    # Latest document received
    docreceived_subquery04 = (
        documentsreceived.groupBy("CaseNo")
          .agg(spark_max("ReceivedDocumentId").alias("max_ReceivedDocumentId"))
    )

    # SA previous status
    sa_prev_subquery05 = (
        status.filter((col("casestatus").isNull()) | (col("casestatus") != '46'))
          .groupBy("CaseNo")
          .agg(spark_max("StatusId").alias("Prev_ID"))
    )

    # Final join
    result = (
        appealcase.alias("ac")
        .join(s_subquery01.alias("s"), col("ac.CaseNo") == col("s.CaseNo"), "left")
        .join(status.alias("t"), (col("t.CaseNo") == col("s.CaseNo")) & (col("t.StatusId") == col("s.max_ID")), "left")
        .join(caselist.alias("cl"), col("t.StatusId") == col("cl.StatusId"), "left")
        .join(prev_subquery02.alias("prev"), col("ac.CaseNo") == col("prev.CaseNo"), "left")
        .join(status.alias("st"), (col("st.CaseNo") == col("prev.CaseNo")) & (col("st.StatusId") == col("prev.Prev_ID")), "left")
        .join(ut_subquery03.alias("ut"), col("ac.CaseNo") == col("ut.CaseNo"), "left")
        .join(status.alias("us"), (col("us.CaseNo") == col("ut.CaseNo")) & (col("us.StatusId") == col("ut.UT_ID")), "left")
        .join(docreceived_subquery04.alias("doc_received"), col("ac.CaseNo") == col("doc_received.CaseNo"), "left")
        .join(documentsreceived.alias("dr"), (col("dr.CaseNo") == col("doc_received.CaseNo")) & (col("dr.ReceivedDocumentId") == col("doc_received.max_ReceivedDocumentId")), "left")
        .join(sa_prev_subquery05.alias("sa_prev"), col("ac.CaseNo") == col("sa_prev.CaseNo"), "left")
        .join(status.alias("sa"), (col("sa.CaseNo") == col("sa_prev.CaseNo")) & (col("sa.StatusId") == col("sa_prev.Prev_ID")), "left")
        .join(filelocation.alias("fl"), col("ac.CaseNo") == col("fl.CaseNo"), "left")
        .join(balancedue.alias("bd"), col("ac.CaseNo") == col("bd.CaseNo"), "left")
        .join(representation.alias("r"), col("ac.CaseNo") == col("r.CaseNo"), "left")
        .filter((col("ac.CaseType") == 1) & (~col("fl.DeptId").isin(519, 520)))
        .select(
            "ac.CaseNo", 
            # "ac.CasePrefix",
            when(
                (col("ac.CasePrefix").isin("EA", "HU")) &
                (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(0, 109, 99, 104, 82, 121)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 1) & (col("sa.CaseStatus") == '10'))
                ) &
                (
                    (col("dr.ReceivedDocumentId") <= 2) |
                    (col("dr.ReceivedDocumentId").isNull())
                ) &
                (col("bd.BalanceDue") > 0), "paymentPending"
            ).when(
                (col("ac.CasePrefix").isin("EA", "HU")) &
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '10') & (col("st.Outcome") == 0)) |
                    (col("st.CaseStatus").isNull())
                ) &
                (
                    (col("dr.ReceivedDocumentId") <= 2) |
                    (col("dr.ReceivedDocumentId").isNull())
                ) &
                (col("bd.BalanceDue") > 0), "paymentPending"
            ).when(
                (
                    (col("ac.CasePrefix").isin("EA", "HU") & ((col("bd.BalanceDue") <= 0) | col("bd.BalanceDue").isNull())) |
                    col("ac.CasePrefix").isin("PA", "DC", "RP", "DA", "IA", "LE", "LP", "LR", "LD", "LH")
                ) &
                (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(0, 109, 99, 104, 82, 121)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 1) & (col("sa.CaseStatus") == '10'))
                ) &
                (
                    (col("dr.ReceivedDocumentId") <= 2) |
                    (col("dr.ReceivedDocumentId").isNull())
                ), "appealSubmitted"
            ).when(
                (
                    (col("ac.CasePrefix").isin("EA", "HU") & ((col("bd.BalanceDue") <= 0) | col("bd.BalanceDue").isNull())) |
                    col("ac.CasePrefix").isin("PA", "DC", "RP", "DA", "IA", "LE", "LP", "LR", "LD", "LH")
                ) &
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '10') & (col("st.Outcome") == 0)) |
                    (col("st.CaseStatus").isNull())
                ) &
                (
                    (col("dr.ReceivedDocumentId") <= 2) |
                    (col("dr.ReceivedDocumentId").isNull())
                ), "appealSubmitted"
            ).when(
                (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(0, 109, 99, 121, 82, 104)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 1) & (col("sa.CaseStatus") == '10'))
                ) &
                (col("dr.ReceivedDocumentId") == 3) &
                (col("dr.DateReceived").isNull()), "awaitingRespondentEvidence(a)"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '10') & (col("st.Outcome") == 0)) |
                    (col("st.CaseStatus").isNull())
                ) &
                (col("dr.ReceivedDocumentId") == 3) &
                (col("dr.DateReceived").isNull()), "awaitingRespondentEvidence(a)"
            ).when(
                (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & (col("t.Outcome") == 0)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 1) & (col("sa.CaseStatus") == '10'))
                ) &
                (col("dr.ReceivedDocumentId") == 3) &
                (col("dr.DateReceived").isNotNull()), "awaitingRespondentEvidence(b)"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '10') & (col("st.Outcome") == 0)) |
                    (col("st.CaseStatus").isNull())
                ) &
                (col("dr.ReceivedDocumentId") == 3) &
                (col("dr.DateReceived").isNotNull()), "awaitingRespondentEvidence(b)"
            ).when(
                (
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(121, 109, 99, 82, 104) &
                    (col("dr.ReceivedDocumentId") == 3) & col("dr.DateReceived").isNotNull()) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(0, 27, 39, 50, 89))
                ) &
                (col("r.Representation") == "LR"), "caseUnderReview"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '26') & col("st.Outcome").isin(0, 27, 39, 50, 89))
                ) &
                (col("r.Representation") == "LR"), "caseUnderReview"
            ).when(
                (
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(121, 109, 99, 82, 104) &
                    (col("dr.ReceivedDocumentId") == 3) & col("dr.DateReceived").isNotNull()) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(0, 27, 39, 50, 89))
                ) &
                (col("r.Representation") == "AIP"), "reasonsForAppealSubmitted"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '26') & col("st.Outcome").isin(0, 27, 39, 50, 89))
                ) &
                (col("r.Representation") == "AIP"), "reasonsForAppealSubmitted"
            ).when(
                (
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(40, 52)) |
                    ((col("t.CaseStatus").isin("37", "38")) & col("t.Outcome").isin(39, 40, 37, 50, 27, 5)) |
                    ((col("t.CaseStatus").isin("37", "38")) & (col("t.Outcome") == 0) & col("t.KeyDate").isNull())
                ), "listing"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus") == '26') & col("st.Outcome").isin(40, 52))
                ), "listing"
            ).when(
                (col("t.CaseStatus").isin("37", "38")) & (col("t.Outcome") == 0) &
                (col("t.KeyDate") > current_date()) &
                (col("t.DecisionReserved") != 1) &
                (col("cl.ListId").isNotNull()), "prepareForHearing"
            ).when(
                (col("t.CaseStatus").isin("37", "38")) & (col("t.Outcome") == 0) &
                (col("t.KeyDate").isNotNull()) &
                (col("t.DecisionReserved") == 1), "decision"
            ).when(
                (
                    ((col("t.CaseStatus").isin("46", "39")) & (col("t.Outcome") == 86)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 1) & (col("sa.CaseStatus") != 10))
                ), "decided(b)"
            ).when(
                (col("t.CaseStatus") == '39') & (col("t.Outcome") == 0) & (col("t.AdjudicatorId") == 0), "fptaSubmitted(a)"
            ).when(
                (col("t.CaseStatus") == '39') & (col("t.Outcome") == 0) & (col("t.AdjudicatorId") != 0), "ftpaSubmitted(b)"
            ).when(
                (
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 31) & col("sa.CaseStatus").isin("37", "38")) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(1, 2)) |
                    ((col("t.CaseStatus").isin("37", "38")) & col("t.Outcome").isin(1, 2)) |
                    ((col("t.CaseStatus") == '52') & col("t.Outcome").isin(91, 95) & col("st.CaseStatus").isin("37", "38", "17")) |
                    ((col("t.CaseStatus") == '36') & col("t.Outcome").isin(1, 2, 25) & col("st.CaseStatus").isin("37", "38", "17"))
                ), "decided(a)"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus").isin("37", "38")) & col("st.Outcome").isin(1, 2)) |
                    (col("st.CaseStatus") == 17)
                ), "decided(a)"
            ).when(
                (
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 31) & (col("sa.CaseStatus") == 39)) |
                    ((col("t.CaseStatus") == '39') & col("t.Outcome").isin(30, 31, 14)) |
                    ((col("t.CaseStatus") == '52') & col("t.Outcome").isin(91, 95) & (col("st.CaseStatus") == 39)) |
                    ((col("t.CaseStatus") == '36') & col("t.Outcome").isin(1, 2, 25) & (col("st.CaseStatus") == 39))
                ), "ftpaDecided"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    (col("st.CaseStatus") == '39') & col("st.Outcome").isin(31, 30, 14)
                ), "ftpaDecided"
            ).when(
                (
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(80, 122, 25, 120, 2, 105, 13)) |
                    ((col("t.CaseStatus") == '46') & (col("t.Outcome") == 31) & col("sa.CaseStatus").isin(10, 51, 52)) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(80, 13, 25)) |
                    ((col("t.CaseStatus").isin("37", "38")) & col("t.Outcome").isin(80, 13, 25, 72, 125)) |
                    ((col("t.CaseStatus") == '39') & (col("t.Outcome") == 25)) |
                    ((col("t.CaseStatus") == '51') & col("t.Outcome").isin(94, 93)) |
                    (((col("t.CaseStatus") == '52') & col("t.Outcome").isin(91, 95) & (~col("st.CaseStatus").isin("37", "38", "39", "17"))) | col("st.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '36') & col("t.Outcome").isin(1, 2, 25) & (~col("st.CaseStatus").isin("37", "38", "39", "17")))
                ), "ended"
            ).when(
                (col("t.CaseStatus").isin("50", "52", "36")) & (col("t.Outcome") == 0) &
                (
                    ((col("st.CaseStatus").isin("10", "26")) & col("st.Outcome").isin(80, 122, 2, 105, 120, 25)) |
                    (col("st.CaseStatus") == '51')
                ), "ended"
            ).when(
                (col("t.CaseStatus").isin("40", "41", "42", "43", "44", "53")) & (col("t.Outcome") == 86), "remitted"
            ).otherwise("Not Sure").alias("TargetState"),
            #################################################
            when(
                (col("t.CaseStatus").isin('40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33')) & (col("t.Outcome") == 86),
                'CCD'
            ).when(
                (col("t.CaseStatus").isin('40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33')) & (col("t.Outcome") == 0),
                'UT Active'
            ).when(
                (col("ac.CasePrefix").isin('VA', 'AA', 'AS', 'CC', 'HR', 'HX', 'IM', 'NS', 'OA', 'OC', 'RD', 'TH', 'XX')) &
                (col("t.CaseStatus").isin('40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33')),
                'UT Overdue'
            ).when(
                col("ac.CasePrefix").isin('VA', 'AA', 'AS', 'CC', 'HR', 'HX', 'IM', 'NS', 'OA', 'OC', 'RD', 'TH', 'XX'),
                'FT Overdue'
            ).when(
                (col("us.CaseStatus").isNotNull()) & (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(0, 109, 104, 82, 99, 121, 27, 39)) |
                    ((col("t.CaseStatus") == '46') & col("t.Outcome").isin(1, 86)) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(0, 27, 39, 50, 40, 52, 89)) |
                    ((col("t.CaseStatus").isin('37', '38')) & col("t.Outcome").isin(39, 40, 37, 50, 27, 0, 5, 52)) |
                    ((col("t.CaseStatus") == '39') & col("t.Outcome").isin(0, 86)) |
                    ((col("t.CaseStatus") == '50') & (col("t.Outcome") == 0)) |
                    ((col("t.CaseStatus").isin('52', '36')) & (col("t.Outcome") == 0) & col("st.DecisionDate").isNull())
                ),
                'CCD'
            ).when(
                (
                    col("ac.CasePrefix").isin('DA', 'DC', 'EA', 'HU', 'PA', 'RP') |
                    (col("ac.CasePrefix").isin('LP', 'LR', 'LD', 'LH', 'LE', 'IA') & col("ac.HOANRef").isNull())
                ) & (
                    (col("t.CaseStatus").isNull()) |
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(0, 109, 104, 82, 99, 121, 27, 39)) |
                    ((col("t.CaseStatus") == '46') & col("t.Outcome").isin(1, 86)) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(0, 27, 39, 50, 40, 52, 89)) |
                    ((col("t.CaseStatus").isin('37', '38')) & col("t.Outcome").isin(39, 40, 37, 50, 27, 0, 5, 52)) |
                    ((col("t.CaseStatus") == '39') & col("t.Outcome").isin(0, 86)) |
                    ((col("t.CaseStatus") == '50') & (col("t.Outcome") == 0)) |
                    ((col("t.CaseStatus").isin('52', '36')) & (col("t.Outcome") == 0) & col("st.DecisionDate").isNull())
                ),
                'CCD'
            ).when(
                (
                    col("ac.CasePrefix").isin('DA', 'DC', 'EA', 'HU', 'PA', 'RP') |
                    (col("ac.CasePrefix").isin('LP', 'LR', 'LD', 'LH', 'LE', 'IA') & col("ac.HOANRef").isNull()) |
                    (
                        col("ac.CasePrefix").isin('LP', 'LR', 'LD', 'LH', 'LE', 'IA') & col("ac.HOANRef").isNotNull() &
                        col("us.CaseStatus").isNotNull() &
                        (add_months(col("us.DecisionDate"), 60) < add_months(col("t.DecisionDate"), 24))
                    )
                ) & (
                    ((col("t.CaseStatus") == '10') & col("t.Outcome").isin(13, 80, 122, 25, 120, 2, 105, 119)) |
                    ((col("t.CaseStatus") == '46') & col("t.Outcome").isin(31, 2, 50)) |
                    ((col("t.CaseStatus") == '26') & col("t.Outcome").isin(80, 13, 25, 1, 2)) |
                    ((col("t.CaseStatus").isin('37', '38')) & col("t.Outcome").isin(1, 2, 80, 13, 25, 72, 14, 125)) |
                    ((col("t.CaseStatus") == '39') & col("t.Outcome").isin(30, 31, 25, 14, 80)) |
                    ((col("t.CaseStatus") == '51') & col("t.Outcome").isin(94, 93)) |
                    ((col("t.CaseStatus") == '52') & col("t.Outcome").isin(91, 95) & ((~col("st.CaseStatus").isin('37', '38', '39', '17', '40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33')) | col("st.CaseStatus").isNull())) |
                    ((col("t.CaseStatus") == '36') & (col("t.Outcome") == 25) & ((~col("st.CaseStatus").isin('40', '41', '42', '43', '44', '45', '53', '27', '28', '29', '34', '32', '33')) | col("st.CaseStatus").isNull()))
                ) & (
                    add_months(col("t.DecisionDate"), 6) >= current_date()
                ),
                'CCD'
            ).when(
                (
                    (col("ac.CasePrefix").isin('DA', 'DC', 'EA', 'HU', 'PA', 'RP') & col("us.CaseStatus").isNull()) |
                    (col("ac.CasePrefix").isin('LP', 'LR', 'LD', 'LH', 'LE', 'IA') & col("ac.HOANRef").isNull()) |
                    (
                        col("ac.CasePrefix").isin('LP', 'LR', 'LD', 'LH', 'LE', 'IA') & col("ac.HOANRef").isNotNull() &
                        col("us.CaseStatus").isNotNull() &
                        (add_months(col("us.DecisionDate"), 60) < add_months(col("t.DecisionDate"), 24))
                    )
                ) & (
                    ((col("t.CaseStatus").isin('52', '36')) & (col("t.Outcome") == 0) & col("st.DecisionDate").isNotNull()) |
                    ((col("t.CaseStatus") == '36') & col("t.Outcome").isin(1, 2, 50, 108)) |
                    ((col("t.CaseStatus") == '52') & col("t.Outcome").isin(91, 95) & col("st.CaseStatus").isin('37', '38', '39', '17'))
                ) & (
                    add_months(col("t.DecisionDate"), 6) >= current_date()
                ),
                'CCD'
            ).otherwise("Other").alias("HighLevelSegment")

        )
    )

    result = result.filter(col('HighLevelSegment') == 'CCD')

    return result

## Silver DLT Tables Creation

### Transformation M1 silver_appealcase_detail

In [0]:
@dlt.table(
    name="silver_appealcase_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_appealcase_detail"
)
def silver_appealcase_detail():
    from pyspark.sql.window import Window
    from pyspark.sql.functions import row_number, col, min, size, when

    try:
        appeal_case_raw = dlt.read("bronze_appealcase_crep_rep_floc_cspon_cfs")
        segmentation_states = dlt.read("stg_segmentation_states")
        countries_nationality = dlt.read("bronze_countries_nationality")
        hearing_centres = dlt.read("bronze_hearing_centres")
        appeal_category = dlt.read("bronze_appealcategory")
        appeal_type = dlt.read("bronze_appealtype")
        representation = dlt.read("stg_representation")
    except:
        appeal_case_raw = spark.table("ariadm_active_appeals.bronze_appealcase_crep_rep_floc_cspon_cfs")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")
        countries_nationality = spark.table("ariadm_active_appeals.bronze_countries_nationality")
        hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")
        appeal_category = spark.table("ariadm_active_appeals.bronze_appealcategory")
        appeal_type = spark.table("ariadm_active_appeals.bronze_appealtype")
        representation = spark.table("ariadm_active_appeals.stg_representation")

    window_spec = Window.partitionBy("CaseNo")
    appeal_case = (
        appeal_case_raw
        .withColumn("min_id", min("CaseFeeSummaryId").over(window_spec))
        .where(col("CaseFeeSummaryId") == col("min_id"))
        .drop("min_id")
    )

    app_cat_agg = (
        appeal_category.groupBy("CaseNo")
        .agg(F.collect_list("CategoryId").alias("CategoryIds"))
    )

    result = (
        appeal_case.alias("ac")
        .join(segmentation_states, on="CaseNo", how="inner")
        .join(countries_nationality.alias("cn"), col("ac.NationalityId") == col("cn.NationalityId"), how="left")
        .join(hearing_centres.alias("hc"), col("ac.CentreId") == col("hc.CentreId"), how="left")
        .join(app_cat_agg.alias("app"), col("ac.CaseNo") == col("app.CaseNo"), how="left")
        .join(representation.alias("rep"), col("ac.CaseNo") == col("rep.CaseNo"), how="left")
        .withColumn(
            "MappedCCDAppealType",
            when(
                (col("ac.CasePrefix") == "EA") &
                (
                    (~F.array_contains(col("app.CategoryIds"), 47)) |
                    (~F.array_contains(col("app.CategoryIds"), 48)) |
                    (~F.array_contains(col("app.CategoryIds"), 49))
                ),
                "EU"
            ).when(
                (col("ac.CasePrefix") == "EA") &
                (
                    F.array_contains(col("app.CategoryIds"), 47) |
                    F.array_contains(col("app.CategoryIds"), 48) |
                    F.array_contains(col("app.CategoryIds"), 49)
                ),
                "EA"
            ).when(col("ac.CasePrefix") == "DA", "EA")
            .when(col("ac.CasePrefix") == "DC", "DC")
            .when(col("ac.CasePrefix") == "HU", "HU")
            .when(col("ac.CasePrefix") == "IA", "HU")
            .when(col("ac.CasePrefix") == "LD", "DC")
            .when(col("ac.CasePrefix") == "LE", "EA")
            .when(col("ac.CasePrefix") == "LH", "HU")
            .when(col("ac.CasePrefix") == "LP", "PA")
            .when(col("ac.CasePrefix") == "LR", "RP")
            .when(col("ac.CasePrefix") == "PA", "PA")
            .when(col("ac.CasePrefix") == "RP", "RP")
            .otherwise(None)
        )
        .join(
            appeal_type.alias("at"),
            col("MappedCCDAppealType") == col("at.CCDAppealType"),
            how="left"
        )
        .withColumn(
            "hearingCentreDynamicList",
            when(
                col("hc.locationCode").isNotNull() & col("hc.locationLabel").isNotNull(),
                F.struct(
                    F.struct(
                        col("hc.locationCode").alias("code"),
                        col("hc.locationLabel").alias("label")
                    ).alias("value"),
                    F.array(
                        F.struct(F.lit("227101").alias("code"), F.lit("Newport Tribunal Centre - Columbus House").alias("label")),
                        F.struct(F.lit("231596").alias("code"), F.lit("Birmingham Civil And Family Justice Centre").alias("label")),
                        F.struct(F.lit("28837").alias("code"), F.lit("Harmondsworth Tribunal Hearing Centre").alias("label")),
                        F.struct(F.lit("366559").alias("code"), F.lit("Atlantic Quay - Glasgow").alias("label")),
                        F.struct(F.lit("366796").alias("code"), F.lit("Newcastle Civil And Family Courts And Tribunals Centre").alias("label")),
                        F.struct(F.lit("386417").alias("code"), F.lit("Hatton Cross Tribunal Hearing Centre").alias("label")),
                        F.struct(F.lit("512401").alias("code"), F.lit("Manchester Tribunal Hearing Centre - Piccadilly Exchange").alias("label")),
                        F.struct(F.lit("649000").alias("code"), F.lit("Yarls Wood Immigration And Asylum Hearing Centre").alias("label")),
                        F.struct(F.lit("698118").alias("code"), F.lit("Bradford Tribunal Hearing Centre").alias("label")),
                        F.struct(F.lit("765324").alias("code"), F.lit("Taylor House Tribunal Hearing Centre").alias("label"))
                    ).alias("list_items")
                )
            ).otherwise(None)
        )
        .withColumn(
            "caseManagementLocationRefData",
            when(
                col("hc.locationCode").isNotNull() & col("hc.locationLabel").isNotNull(),
                F.struct(
                    F.lit("1").alias("region"),
                    F.struct(
                        F.struct(
                            col("hc.locationCode").alias("code"),
                            col("hc.locationLabel").alias("label")
                        ).alias("value"),
                        F.array(
                            F.struct(F.lit("227101").alias("code"), F.lit("Newport Tribunal Centre - Columbus House").alias("label")),
                            F.struct(F.lit("231596").alias("code"), F.lit("Birmingham Civil And Family Justice Centre").alias("label")),
                            F.struct(F.lit("28837").alias("code"), F.lit("Harmondsworth Tribunal Hearing Centre").alias("label")),
                            F.struct(F.lit("366559").alias("code"), F.lit("Atlantic Quay - Glasgow").alias("label")),
                            F.struct(F.lit("366796").alias("code"), F.lit("Newcastle Civil And Family Courts And Tribunals Centre").alias("label")),
                            F.struct(F.lit("386417").alias("code"), F.lit("Hatton Cross Tribunal Hearing Centre").alias("label")),
                            F.struct(F.lit("512401").alias("code"), F.lit("Manchester Tribunal Hearing Centre - Piccadilly Exchange").alias("label")),
                            F.struct(F.lit("649000").alias("code"), F.lit("Yarls Wood Immigration And Asylum Hearing Centre").alias("label")),
                            F.struct(F.lit("698118").alias("code"), F.lit("Bradford Tribunal Hearing Centre").alias("label")),
                            F.struct(F.lit("765324").alias("code"), F.lit("Taylor House Tribunal Hearing Centre").alias("label"))
                        ).alias("list_items")
                    ).alias("baseLocation")
                )
            ).otherwise(None)
        )
        .select(
            "ac.*",
            col("TargetState").alias("dv_targetState"),
            col("cn.countryCode").alias("lu_countryCode"),
            col("cn.appellantNationalitiesDescription").alias("lu_appellantNationalitiesDescription"),
            col("hc.hearingCentre").alias("lu_hearingCentre"),
            col("hc.staffLocation").alias("lu_staffLocation"),
            col("hc.caseManagementLocation").alias("lu_caseManagementLocation"),
            col("hc.locationCode").alias("lu_locationCode"),
            col("hc.locationLabel").alias("lu_locationLabel"),
            col("hc.selectedHearingCentreRefData").alias("lu_selectedHearingCentreRefData"),
            col("hc.listCaseHearingCentre").alias("lu_listCaseHearingCentre"),
            col("hc.prevFileLocation").alias("lu_prevFileLocation"),
            col("hc.applicationChangeDesignatedHearingCentre").alias("lu_applicationChangeDesignatedHearingCentre"),
            col("at.CCDAppealType").alias("dv_CCDAppealType"),
            col("at.appealType").alias("lu_appealType"),
            col("at.hmctsCaseCategory").alias("lu_hmctsCaseCategory"),
            col("at.appealTypeDescription").alias("lu_appealTypeDescription"),
            col("rep.representation").alias("dv_representation"),
            col("at.caseManagementCategory").alias("lu_caseManagementCategory"),
            col("hearingCentreDynamicList").alias("dv_hearingCentreDynamicList"),
            col("caseManagementLocationRefData").alias("dv_caseManagementLocationRefData")
        )
    ).distinct()
    return result
#

### Transformation M2 silver_caseapplicant_detail

In [0]:
@dlt.table(
    name="silver_caseapplicant_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_caseapplicant_detail"
)
def silver_caseapplicant_detail():
    try:
        # Read the AppealCase, CaseRep, and CountriesPostal DLT tables
        app = dlt.read("bronze_appealcase_caseappellant_appellant")
        segmentation_states = dlt.read("stg_segmentation_states")
        countries_postal = dlt.read("bronze_countries_postal")
    except:
        app = spark.table("ariadm_active_appeals.bronze_appealcase_caseappellant_appellant")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")
        countries_postal = spark.table("ariadm_active_appeals.bronze_countries_postal")

    result = (
        app.alias("a")
        .join(segmentation_states, on="CaseNo", how="inner")
        .join(countries_postal.alias("cp"), col("a.AppellantCountryId") == col("cp.AppellantCountryId"), how="left")
        .select(
            "a.*", 
            col("TargetState").alias("dv_targetState"), 
            col("cp.countryGovUkOocAdminJ").alias("lu_countryGovUkOocAdminJ")
        )
    )
    return result

### Transformation M3 silver_status_detail

In [0]:
@dlt.table(
    name="silver_status_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_status_detail"
)
def silver_status_detail():
    try:
        # Read the AppealCase, CaseRep, and HearingCentres DLT tables
        appeal_case = dlt.read("bronze_status_htype_clist_list_ltype_court_lsitting_adj")
        segmentation_states = dlt.read("stg_segmentation_states")
        hearing_centres = dlt.read("bronze_hearing_centres")
    except:
        appeal_case = spark.table("ariadm_active_appeals.bronze_status_htype_clist_list_ltype_court_lsitting_adj")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")
        hearing_centres = spark.table("ariadm_active_appeals.bronze_hearing_centres")

    result = (
        appeal_case.alias("ac")
        .join(segmentation_states, on="CaseNo", how="inner")
        .join(hearing_centres.alias("hc"), col("ac.CentreId") == col("hc.CentreId"), how="left")
        .withColumn("tr_Notes", regexp_replace(regexp_replace(trim(col("ac.Notes")), '[\r\n]+', ' '), ' +', ' '))
        .select(
            "ac.*",
            col("TargetState").alias("dv_targetState"),
            col("hc.hearingCentre").alias("lu_hearingCentre"),
            col("hc.staffLocation").alias("lu_staffLocation"),
            col("hc.caseManagementLocation").alias("lu_caseManagementLocation"),
            col("hc.locationCode").alias("lu_locationCode"),
            col("hc.locationLabel").alias("lu_locationLabel"),
            col("hc.selectedHearingCentreRefData").alias("lu_selectedHearingCentreRefData"),
            col("hc.listCaseHearingCentre").alias("lu_listCaseHearingCentre"),
            col("hc.applicationChangeDesignatedHearingCentre").alias("lu_applicationChangeDesignatedHearingCentre"),
            col("tr_Notes")
        )
    )
    return result

### Transformation M4 silver_transaction_detail

In [0]:
@dlt.table(
    name="silver_transaction_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_transaction_detail"
)
def silver_transaction_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        tran = dlt.read("bronze_appealcase_transaction_transactiontype")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        tran = spark.table("ariadm_active_appeals.bronze_appealcase_transaction_transactiontype")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = tran.alias("t").join(segmentation_states, on="CaseNo", how="inner").select("t.*",col("TargetState").alias("dv_targetState"))
    return result

### Transformation M5 silver_link_detail

In [0]:
@dlt.table(
    name="silver_link_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_link_detail"
)
def silver_link_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        link = dlt.read("bronze_appealcase_link_linkdetail")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        link = spark.table("ariadm_active_appeals.bronze_appealcase_link_linkdetail")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = link.alias("l").join(segmentation_states, on="CaseNo", how="inner").select("l.*",col("TargetState").alias("dv_targetState"))
    return result

### Transformation M6 silver_adjudicator_detail

In [0]:
@dlt.table(
    name="silver_adjudicator_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_adjudicator_detail"
)
def silver_adjudicator_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        adj = dlt.read("bronze_caseadjudicator_adjudicator")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        adj = spark.table("ariadm_active_appeals.bronze_caseadjudicator_adjudicator")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = adj.alias("adj").join(segmentation_states, on="CaseNo", how="inner").select("adj.*",col("TargetState").alias("dv_targetState"))
    return result

### Transformation silver_appealcategory_detail

In [0]:
@dlt.table(
    name="silver_appealcategory_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_appealcategory_detail"
)
def silver_appealcategory_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        ac = dlt.read("bronze_appealcategory")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        ac = spark.table("ariadm_active_appeals.bronze_appealcategory")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = ac.alias("ac").join(segmentation_states, on="CaseNo", how="inner").select("ac.*",col("TargetState").alias("dv_targetState"))
    return result

### Transformation silver_documentsreceived_detail

In [0]:
@dlt.table(
    name="silver_documentsreceived_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_documentsreceived_detail"
)
def silver_documentsreceived_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        dr = dlt.read("bronze_documentsreceived")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        dr = spark.table("ariadm_active_appeals.bronze_documentsreceived")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = dr.alias("dr").join(segmentation_states, on="CaseNo", how="inner").select("dr.*",col("TargetState").alias("dv_targetState"))
    return result

### Transformation silver_history_detail

In [0]:
@dlt.table(
    name="silver_history_detail",
    comment="DLT table to determine if each case is AIP or LR based on CaseRep information.",
    path=f"{silver_path}/silver_history_detail"
)
def silver_history_detail():
    try:
        # Read the AppealCase and CaseRep DLT tables
        dr = dlt.read("bronze_history")
        segmentation_states = dlt.read("stg_segmentation_states")
    except:
        dr = spark.table("ariadm_active_appeals.bronze_history")
        segmentation_states = spark.table("ariadm_active_appeals.stg_segmentation_states")

    result = dr.alias("dh").join(segmentation_states, on="CaseNo", how="inner").select("dh.*",col("TargetState").alias("dv_targetState"))
    return result

## Exit Notebook with Success Message

In [0]:
dbutils.notebook.exit("Notebook completed successfully")

### Appendix

In [0]:
# %sql
# drop schema if exists ariadadm_active_appeals cascade